# AdaBoost y Gradient boosting con árboles en un dataset de detección de spam

Árboles de decisión y todos los ensembles propuestos funcionan en la práctica muy bien en datos tabulares como puede ser el problema de reconocimiento de spam en mails.

Se propone pues emplear el dataset id=44 de openml de detección de Spam. Son un total de 4601 muestras con 57 características.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

## Descarga del dataset Spam
X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)
print(X.shape)

## Partición train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)



(4601, 57)


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


## AdaBoost

Probemos el clasificador de AdaBoost

In [34]:
from sklearn.ensemble import AdaBoostClassifier

ab = AdaBoostClassifier()

acc=ab.fit(X_train, y_train).score(X_test, y_test)

print(f'Precisión: {acc:.1%}')


Precisión: 93.8%


**Ejercicio** Teniendo en cuenta la documentación [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html) prueba mediante grid search a encontrar mejores parámetros.

## GradientBoostingClassifier

Probemos el clasificador Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()

acc=gb.fit(X_train, y_train).score(X_test, y_test)

print(f'Precisión: {acc:.1%}')


Precisión: 95.2%


Los principales parámetros del GradientBoosting son:
* n_estimators (100)
* max_depth (3)
* max_features

**Ejercicio** prueba diferentes valores (lógicos) de estos parámetros mediante GridSearch.

### Acelerar la búsqueda de parámetros: RandomizedSearch

Un problema recurrente que tenemos en estos experimentos es el coste temporal de encontrar los mejores parámetros mediante GridSearch. Una alternativa que reduce dicho coste temporal es el [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

**Ejercicio** Implementa la misma búsqueda de valores que has realizado con GridSearch pero ahora con RandomizedSearch con n_iter=10 para limitar el número de pruebas a 10.

### Mejorar la búsqueda de parámetros: BayesianOptimization

Optimización Bayesiana es una técnica avanzada de búsqueda de parámetros. Para poder emplearla hay que instalar la librería scikit-optimize. El uso de dicha técnica es similar a las ya empleadas con la diferencia que el rango de los parámetros hay que definirlos teniendo en cuenta además el tipo de los mismos: real, entero o categórico. Además los valores a probar no se listan explícitamente si no que se especifica un rango dentro del cual el algoritmo escogerá los valores.

La ejecución puede resultar más lenta pero se pueden explorar más parámetros, de hecho se especifica un intervalo en lugar de valores concretos.



In [ ]:
!pip install scikit-optimize

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, IntegerP2_S3.ipynb
BS = BayesSearchCV(GradientBoostingClassifier(), G, scoring='accuracy', n_iter=10, refit=True, cv=5, verbose=10)

acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {BS.best_params_}')


## HistGradientBoostingClassifier
Un problema del clasificador GradientBoosting implementado en sklearn es su velocidad. Es un clasificador muy lento de entrenar. Por ello sklearn propone otro tipo de algoritmo de GradientBoosting que soporta paralelismo con OMP además de otras mejoras computacionales basadas en la discretización de las componentes mediante un histograma.

Este otro algoritmo se denomina **HistGradientBoostingClassifier**. Su tiempo de ejecución es mucho mejor. Además se pueden obtener mejores resultados.



In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier()

acc=hgb.fit(X_train, y_train).score(X_test, y_test)

print(f'Precisión: {acc:.1%}')

Precisión: 95.5%


**Ejercicio** Analiza en la documentación [HistGradientBoosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html) los parámetros más relevantes y realiza una búsqueda de parámetros para obtener el mejor clasificador. A la hora de establecer los valores de los parámetros sería interesante fijarnos en los valores por defecto que se han empleado en el ejercicio anterior y poder explorar alrededor de dichos valores por defecto para conseguir mejores resultados.

